In [9]:
import pickle

from prometheus_client import CollectorRegistry, Summary, Info
from prometheus_client.exposition import write_to_textfile

def toSnakeCase(string):
  return string.replace(" ", "_")

def PublishSamples(samples, registry):
    
  samples = list(samples)
  
  # Metrics
  metric_summaries = dict()  # metric key to Summary
  metric_keys = set(toSnakeCase(sample['metric']) for sample in samples)  # all metric keys in snake case
  for metric_key in metric_keys:
      metric_summaries[metric_key] = Summary(name=toSnakeCase(metric_key),
                                             documentation=metric_key + ' values',
                                             labelnames=['test', 'unit'],
                                             registry=registry)

  # Going through samples
  for sample in samples:
      
    # Metric value
    metric_summaries[toSnakeCase(sample['metric'])]\
        .labels(test=sample['test'], unit=sample['unit'])\
        .observe(sample['value'])
    
    # Writing to file
    write_to_textfile('results.prom', registry)

with open('results.pkl', 'rb') as fp:
    registry = CollectorRegistry()
    # metadata = Info('distributed_k8s', 'Samples\' metadata', registry=registry)
    PublishSamples(pickle.load(fp), registry)